<center>
    <header>
        <h1>Analisi dello Stack TCP-IP</h1>
        <h2>Marco Bondaschi, Mauro Conte</h2>
        <h4>Laboratorio di Telecomunicazioni</h4>
        <h5>Università Degli Studi Di Brescia</h5>
        <h6>22/05/2017</h6>
    </header>
</center>
<hr><hr>

<h2>Descrizione</h2>

Con quest'ultima esperienza abbiamo osservato ed analizzato il funzionamento dello stack protocollare TCP-IP, sperimentando i diversi protocolli che esso utilizza per gestire le più svariate attività, e verificando le caratteristiche dei vari pacchetti che entrano in gioco durante la comunicazione tra due terminali. Grazie a questa attività abbiamo quindi sperimentato le diverse opzioni offerte da un classico analizzatore di protocollo, similmente a quanto fatto durante la precedente esercitazione.


<h2>Strumenti</h2>

Gli strumenti che abbiamo utilizzato per svolgere questa esperienza sono:
<ul>
<li>I nostri due computer Windows, su cui è stato installato l'analizzatore di protocollo <i>Wireshark</i>;</li>
<li>I due PC in dotazione al laboratorio, su cui è presente l'analizzatore di protocollo <i>Fluke Protocol Inspector</i>;</li>
<li>Un cavo coassiale con cui collegare tra loro le schede di rete dei due PC;</li>
<li>Il software <i>Microsoft Excel</i> per l'elaborazione dei dati ottenuti dall'analisi di protocollo.</li>
</ul>


<h2>Svolgimento</h2>
<h3>Configurazione della postazione di lavoro</h3>

Per svolgere la prima parte dell'esperienza abbiamo utilizzato i nostri due PC, su cui abbiamo installato <i>Wireshark</i>. Poichè la rete universitaria impediva di inviare pacchetti ping tra i due terminali, abbiamo optato per l'utilizzo di un <i>hotspot</i> tramite un nostro cellulare, a sua volta collegato alla rete mobile. Una volta collegati i due PC all'hotspot, abbiamo potuto procedere con l'esperienza.
